In [7]:
from models.models import *
from utils.utils import *

import torch, torchvision
from torchvision import transforms
from torch.utils.data import DataLoader



# Cifar10

In [26]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [31]:
batch_size = 256
num_workers = 6
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Resnet18

Load train and test set

In [27]:
train_set = torchvision.datasets.CIFAR10(
    root='./data', train=True, transform=transform 
)
heldout_set = torchvision.datasets.CIFAR10(
    root='./data', train=False, transform=transform
)
test_set, val_set = torch.utils.data.random_split(heldout_set, [0.5, 0.5])

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [29]:
tmp = DataLoader(train_set, batch_size=10, shuffle=True, num_workers=num_workers)
images, labels = next(iter(tmp))

torch.Size([10, 3, 32, 32])

Train a resnet18 pretrain model

In [38]:
pretrain = get_resnet18(num_classes=10)
pretrain.to(DEVICE)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(pretrain.parameters())

In [39]:
losses = []
for epoch in (1, 10+1):
    pretrain.train()
    epoch_loss = 0
    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = pretrain(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 999:
            print(f'Epoch {epoch} batch {batch_idx+1} loss: {loss.item():.4f}')
        epoch_loss += loss.item()
    losses.append(epoch_loss)

pretrain.eval()

/home/ngin/PycharmProjects/Machine_Unlearning/.venv/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/ngin/PycharmProjects/Machine_Unlearning/.venv/lib/python3.11/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  